In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import gc

%matplotlib inline

In [2]:
train = pd.read_csv('../training-set.csv',header=None)
test = pd.read_csv('../testing-set.csv',header=None)
train.columns = ['file_id','target']
test.columns = ['file_id','target']

file_df = pd.read_csv('file_df.csv')

data = pd.concat([train,test],axis=0)
data = pd.merge(data,file_df,how='left',on='file_id')

product_cnt = pd.read_csv('../product_cnt.csv')
product_cnt = product_cnt.fillna(0)

cust_prod = pd.read_csv('../cust_prod.csv')
cust_prod.drop('file_freq_sum',axis=1,inplace=True)

hr_int = pd.read_csv('../hr_int.csv')

hour_cnt = pd.read_csv('../hour_interval_cnt.csv')

minute_cnt = pd.read_csv('../min_interval_cnt.csv')

sec_cnt = pd.read_csv('../sec_interval_cnt.csv')

cchrint = pd.read_csv('../hour_interval_cust_cnt.csv')

ccminint = pd.read_csv('../min_interval_cust_cnt.csv')

In [3]:
data = pd.merge(data,product_cnt,how='outer',on='file_id')
data = pd.merge(data,cust_prod,how='outer',on='file_id')
data = pd.merge(data,hr_int,how='outer',on='file_id')
data = pd.merge(data,hour_cnt,how='outer',on='file_id')
data = pd.merge(data,minute_cnt,how='outer',on='file_id')
data = pd.merge(data,sec_cnt,how='outer',on='file_id')
data = pd.merge(data,cchrint,how='outer',on='file_id')
data = pd.merge(data,ccminint,how='outer',on='file_id')

In [4]:
data['hr_int1'] = (data['hr_int1'] / data['C_sum'])
data['hr_int2'] = (data['hr_int2'] / data['C_sum'])
data['hr_int3'] = (data['hr_int3'] / data['C_sum'])
data['hr_int4'] = (data['hr_int4'] / data['C_sum'])

data['hour_interval1'] = (data['hour_interval1'] / data['C_sum'])
data['hour_interval2'] = (data['hour_interval2'] / data['C_sum'])
data['hour_interval3'] = (data['hour_interval3'] / data['C_sum'])
data['hour_interval4'] = (data['hour_interval4'] / data['C_sum'])

data['min_interval1'] = (data['min_interval1'] / data['C_sum'])
data['min_interval2'] = (data['min_interval2'] / data['C_sum'])
data['min_interval3'] = (data['min_interval3'] / data['C_sum'])
data['min_interval4'] = (data['min_interval4'] / data['C_sum'])

data['sec_interval1'] = (data['sec_interval1'] / data['C_sum'])
data['sec_interval2'] = (data['sec_interval2'] / data['C_sum'])
data['sec_interval3'] = (data['sec_interval3'] / data['C_sum'])
data['sec_interval4'] = (data['sec_interval4'] / data['C_sum'])

data['cchrinte1'] = data['cchrinte1']/ data['cust_cnt']
data['cchrinte2'] = data['cchrinte2']/ data['cust_cnt']
data['cchrinte3'] = data['cchrinte3']/ data['cust_cnt']
data['cchrinte4'] = data['cchrinte4']/ data['cust_cnt']

data['ccmininte1'] = data['ccmininte1']/ data['cust_cnt']
data['ccmininte2'] = data['ccmininte2']/ data['cust_cnt']
data['ccmininte3'] = data['ccmininte3']/ data['cust_cnt']
data['ccmininte4'] = data['ccmininte4']/ data['cust_cnt']

data.drop(['C_sum'],axis=1,inplace=True)

In [5]:
np.sum(np.sum(pd.isnull(data)))

0

In [6]:
data.drop(['3c2be6', 'dd8d4a', '26a5d0', '885fab', '0cdb7a', 
           'a310bb', 'fec24f', '0374c4', '05b409', 'cc3a6a', 
           '218578', 'aaa9c8', '8b7f69', '262880', '8452da', 
           '75f310',  '8541a0','e47f04' ,'533133', 'b93794',],axis=1,inplace=True)

In [7]:
data.drop([ 'C1','C2','C3', 'C4', 'C5','C6',   
            'Cm1','Cm2','Cm3','Cm4','Cm6','Cm5','Cm_sum',
            'Cper3', 'Cper2',  'Cper5',  'Cper6',
            'Cmper3','Cmper5','Cmper4', 'Cmper2',
            'C_last','Cm_last','Cmper_last',
            'Cmper_median',   'C_median',
            'Cper_min','C_min','Cmper_min', 'Cm_min',
            'Cmper_max','Cm_max','Cper_max','C_max',
            'C_std', 'Cm_std','Cmper_std', 'Cper_std',
            'Cmper_mean', 'Cper_mean',  'Cm_mean','C_mean',
            'Cpermax_Cpermin', 'Cmpermax_Cmpermin',  'Cmmax_Cmmin', 'Cmax_Cmin',  
            'Cm1_Cm2', 'Cper1_Cper2','Cmper1_Cmper2',  'C1_C2',
            'Cmperfirst_Cmperlast',  'Cmfirst_Cmlast','Cperfirst_Cperlast','Cfirst_Clast'],axis=1,inplace=True)

# mean encoding

In [8]:
meancode = pd.read_csv('cust_gpmean_val.csv')
#meancode = pd.read_csv('cust_gpmean.csv')
meancode = meancode[['file_id','cust_gp0_mean','cust_gp1_mean','cust_gp2_mean','cust_gp4_mean']]
data = pd.merge(data,meancode,how='left',on='file_id')

In [9]:
newfeat = pd.read_csv('qcgp_feat.csv')
data = pd.merge(data,newfeat,how='left',on='file_id')

In [10]:
np.sum(np.sum(pd.isnull(data)))

0

In [11]:
#for testing
df_train = data[data['target']!=0.5] 
df_test  = data[data['target']==0.5]
#######for val

df_test = df_train[(df_train['first_day']>49)&(df_train['first_day']<57)]

df_train = df_train[df_train['first_day']<50]
#######for val

In [12]:
y_train = df_train['target']
y_test = df_test['target']
train = df_train.drop(['file_id','target','first_day','last_day'],axis=1)
test = df_test.drop(['file_id','target','first_day','last_day'],axis=1)

In [13]:
from sklearn import preprocessing
scaler = preprocessing.StandardScaler().fit(train)
value_train = scaler.transform(train)
value_test  = scaler.transform(test)

In [14]:
train = pd.DataFrame(value_train,columns=train.columns)
test = pd.DataFrame(value_test,columns=train.columns)

In [15]:
train.shape , test.shape

((43943, 58), (5392, 58))

# light GBM

In [16]:
import lightgbm as lgb

In [17]:
# create dataset for lightgbm
lgb_train = lgb.Dataset(train, label=y_train)
lgb_eval = lgb.Dataset(test, label=y_test, reference=lgb_train)

In [18]:
# specify your configurations as a dict
params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'auc',
    'is_unbalance':True,
    'min_data_in_leaf':200,
#    'min_sum_hessian_in_leaf':10,
    
    
    'num_leaves': 2**5,
    'learning_rate': 0.01,
    'feature_fraction': 0.7,
    'bagging_fraction': 0.7,
    'bagging_freq': 1,
    'verbose': 0,
    'lambda_l1':0
    ,'lambda_l2':1
}

In [19]:
# train
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=1539,
                ###val
                valid_sets=[lgb_train,lgb_eval],
                ###val
                early_stopping_rounds=100
               )

[1]	training's auc: 0.94026	valid_1's auc: 0.916364
Training until validation scores don't improve for 100 rounds.
[2]	training's auc: 0.946211	valid_1's auc: 0.924026
[3]	training's auc: 0.947402	valid_1's auc: 0.924232
[4]	training's auc: 0.948747	valid_1's auc: 0.925774
[5]	training's auc: 0.949985	valid_1's auc: 0.927623
[6]	training's auc: 0.95046	valid_1's auc: 0.928782
[7]	training's auc: 0.95122	valid_1's auc: 0.930158
[8]	training's auc: 0.954373	valid_1's auc: 0.933699
[9]	training's auc: 0.954447	valid_1's auc: 0.934747
[10]	training's auc: 0.954406	valid_1's auc: 0.934717
[11]	training's auc: 0.954331	valid_1's auc: 0.934448
[12]	training's auc: 0.954513	valid_1's auc: 0.934936
[13]	training's auc: 0.954574	valid_1's auc: 0.934949
[14]	training's auc: 0.956029	valid_1's auc: 0.936713
[15]	training's auc: 0.956054	valid_1's auc: 0.93648
[16]	training's auc: 0.956729	valid_1's auc: 0.937815
[17]	training's auc: 0.956836	valid_1's auc: 0.938052
[18]	training's auc: 0.957279	va

In [21]:
df = pd.DataFrame(gbm.feature_name())
df['feature_importance']=gbm.feature_importance()
df.columns=['feature_name','feature_importance']
df =df.sort_values(by='feature_importance',ascending=False).reset_index(drop=True)


In [ ]:
gbmpredict = gbm.predict(test)
df_test = df_test.copy()
df_test['gbmpredict']=0
df_test = df_test[['file_id','gbmpredict']].copy()
df_test['gbmpredict'] = gbmpredict

In [ ]:
df_test.to_csv('gbmgpmean.csv',index=False)